In [17]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error

In [4]:
# Загружаем данные
data = pd.read_excel('data/data_ford_price.xlsx') 
data.head()

,price,year,condition,cylinders,odometer,title_status,transmission,drive,size,lat,long,weather
0,43900,2016,4,6,43500,clean,automatic,4wd,full-size,36.471500,-82.483400,59.0
1,15490,2009,2,8,98131,clean,automatic,4wd,full-size,40.468826,-74.281734,52.0
2,2495,2002,2,8,201803,clean,automatic,4wd,full-size,42.477134,-82.949564,45.0
3,1300,2000,1,8,170305,rebuilt,automatic,4wd,full-size,40.764373,-82.349503,49.0
4,13865,2010,3,8,166062,clean,automatic,4wd,NaN,49.210949,-123.114720,NaN


In [5]:
# Отберем числовые признаки

data = data[['price','year', 'cylinders', 'odometer', 'lat', 'long', 'weather']]
#Удалим пустые значения
data.dropna(inplace = True)

# Отделим целевую колонку от данных
y = data['price']
x = data.drop(columns='price')

#разделим выборку на трейн и тест
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)

## Отбор признаков. Метод рекурсивного исключения признаков

In [6]:
# Импортируем метод
from sklearn.feature_selection import RFE

estimator = LinearRegression()
# Применяем метод
selector = RFE(estimator, n_features_to_select=3, step=1)
selector = selector.fit(X_train, y_train)
 
#Отобранные признаки 
selector.get_feature_names_out()

array(['year', 'cylinders', 'lat'], dtype=object)

In [18]:
# Оставляем в наборе данных только выделенные выше признаки
columns = ['year', 'cylinders', 'lat']
X_train_RFE = X_train[columns]
X_test_RFE = X_test[columns]

# Обучаем линейную регрессию на трех отобранных признаках
model = LinearRegression()
model.fit(X_train_RFE, y_train)
y_predicted = model.predict(X_test_RFE)
 
# Оцениваем модель на тесте
mae = mean_absolute_error(y_test, y_predicted)
print('MAE: %.3f' % mae)

r2 = r2_score(y_test, y_predicted)
print('R2-score: %.3f' % r2)

mse = mean_squared_error(y_test, y_predicted)
print('MSE: %.3f' % mse)

MAE: 5096.570
R2-score: 0.576
MSE: 50621735.773


## Метод отбора признаков на основе фильтров

In [15]:
# Импортируем метод
from sklearn.feature_selection import SelectKBest, f_regression

# Применяем метод
selector = SelectKBest(f_regression, k=3)
selector.fit(X_train, y_train)
 
# Отобранные признаки
selector.get_feature_names_out()

array(['year', 'cylinders', 'odometer'], dtype=object)

In [19]:
# Оставляем в наборе данных только выделенные выше признаки
columns = ['year', 'cylinders', 'odometer']
X_train_Kbest = X_train[columns]
X_test_Kbest = X_test[columns]

# Обучаем линейную регрессию на трех отобранных признаках
model = LinearRegression()
model.fit(X_train_Kbest, y_train)
y_predicted = model.predict(X_test_Kbest)
 
# Оцениваем модель на тесте
mae = mean_absolute_error(y_test, y_predicted)
print('MAE: %.3f' % mae)

r2 = r2_score(y_test, y_predicted)
print('R2-score: %.3f' % r2)

mse = mean_squared_error(y_test, y_predicted)
print('MSE: %.3f' % mse)

MAE: 4708.946
R2-score: 0.602
MSE: 47561634.595


### <b>Вывод:</b> по метрикам MAE, R2-score, MSE метод отбора признаков на основе фильтров (SelectKBest) справился с задачей лучше, чем метод рекурсивного исключения признаков (RFE)